<a href="https://colab.research.google.com/github/AVRusakov/FAforRTS/blob/main/FAforRTS2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ниже приведено исследование о связи данных из отчетов публичных компаний российского фондового рынка с изменением стоимости их акций. Данные загружены с сайта smart-lab.ru для компаний входящих в индекс широкого рынка (крупнейшие 100), цена акций каждой комании была скорректирована на выплаченные дивиденты, чтобы учесть полную доходность. В связи с сильным изменением структуры рынка целесообразно разделить периоды до 2021го(включительно) и после 2022го. 

Цели:
1. Сравнить результат инвестирования в индекс ММВБ(IMOEX) и в портфель собранный из отдельных акций. 
2. Сделать выводы о том как выбрать бумаги, имеющие большую доходность.


Пожелания и замечания прошу направить на почту: a.v.rusakov@gmail.com

In [ ]:
#@title Шаг 1. Загрузка модулей и импорт данных

import pandas as pd
import numpy as np


from google.colab import drive
drive.mount('/content/drive')
df_full = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Merged_Annual_adj.csv", sep=',', header=0, index_col=0)



df = df_full.copy()

df_ind = pd.DataFrame(df.loc["Цена скорр.2"].iloc[:,0], dtype=str)  # - индекс(=имя компании) перемещаем в отдельную таблицу
df = pd.DataFrame(df.loc["Цена скорр.2"].iloc[:,1:-1], dtype=float) # отбрасываем имя и LTM, преобразуем тип данных

p = 11 # Период исследования, по умолчанию 11 лет. То есть 2010-2021 годы включительно

# преобразование таблицы, цены за каждый последующий период показаны в отношении к предыдущему периметру ( то есть 0, 2 , 3 , 4 как NaN, NaN, 3/2, 4/3 )
df_a = df.iloc[:,-p:]
df_b = df.iloc[:,-p-1:-1]
df_b.columns = df_a.columns
df = df_a.div(df_b)
df = df.reset_index(drop=True)
#df


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Шаг 2. Перечень возможных вариантов, с учетом срока инвестирования [срок выполнения ~25 сек]
y = 4 # Указать минимальный срок для удержания акций, в течение которого их нельзя продавать. По умолчанию 4.




# df1 это "карта" для таблицы df. Указано количество не нулевых столбцов и номер первого ненулевого.
df1 = df.count(axis=1)
df1 = pd.concat([df.count(axis=1),\
                 pd.DataFrame(df.columns.get_indexer([df.apply(pd.Series.first_valid_index, axis=1)][0]), index=df.apply(pd.Series.first_valid_index, axis=1).index)],\
                 axis = 1,ignore_index=True)

print(df1)


df2 = pd.DataFrame()

# Далее выполняется вполняется нарезка на все возможные варианты для каждой из компаний.
# Например если y=3, то значения цены за 5 лет (1 2 3 4 5) разбиты на варианты (1 2 3) (2 3 4) (3 4 5)(1 2 3 4)(2 3 4 5)(1 2 3 4 5).
# Сложность возрастает значительно при уменьшении параметра y.
for x in range(len(df1[0])):
    #print('x=',x,df1[0][x],df1[1][x]) # номер строки(номер строки=имя компании), количество значений, номер первого ненулевого
    
    if df1[0][x] == y:
        df2 = pd.concat([df2,df.iloc[x]], axis = 1)
    if df1[0][x] > y:
        df2 = pd.concat([df2,df.iloc[x]], axis = 1)
        z = df1[0][x] - y
        for i in range(1,z+1): # размер уменьшения: (3- или 5-) 4,3,2
            #print('i=',i) #количество итераций внутри должно быть count - i + 1
            #print("длина=",df1[0][x]-i)
            #print("кол=",i+1)
            for j in range(i+1): # размер смещения
                #print('j=',j)
                #print(df.loc[x][df1[1][x]+j:df1[1][x]+df1[0][x]-i+j])
                df2 = pd.concat([df2,df.iloc[x][df1[1][x]+j:df1[1][x]+df1[0][x]-i+j]], axis = 1)
    
#print(df2)
df3 = df2.transpose().copy()
df3 # это набор всех возможных вариантов инвестирования, с учетом y

     0   1
0    1  10
1   11   0
2   10   1
3   11   0
4   11   0
..  ..  ..
86  11   0
87   7   4
88  11   0
89  10   1
90   4   7

[91 rows x 2 columns]


,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,1.262669,0.715302,0.447761,0.511111,1.298587,3.940254,0.860961,1.007346,1.485446,1.210634,0.438375
1,1.262669,0.715302,0.447761,0.511111,1.298587,3.940254,0.860961,1.007346,1.485446,1.210634,NaN
1,NaN,0.715302,0.447761,0.511111,1.298587,3.940254,0.860961,1.007346,1.485446,1.210634,0.438375
1,1.262669,0.715302,0.447761,0.511111,1.298587,3.940254,0.860961,1.007346,1.485446,NaN,NaN
1,NaN,0.715302,0.447761,0.511111,1.298587,3.940254,0.860961,1.007346,1.485446,1.210634,NaN
...,...,...,...,...,...,...,...,...,...,...,...
89,NaN,NaN,NaN,NaN,1.302857,2.416667,0.900944,1.124372,NaN,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,2.416667,0.900944,1.124372,1.336451,NaN,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,0.900944,1.124372,1.336451,0.765664,NaN
89,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.124372,1.336451,0.765664,1.633602


In [ ]:
#@title Шаг 3. Расчет эффективности вложения и загрузка функций
df4 = df3.copy()

#df5 это "карта" для таблицы df3: индекс, количество ненулевых столбцов,номер первое ненулевого столбца
df5 = pd.concat([df4.count(axis=1),\
                 pd.DataFrame(df4.columns.get_indexer([df4.apply(pd.Series.first_valid_index, axis=1)][0]),\
                 index=df4.index)], axis = 1,ignore_index=True)

#к карте df5 добавляет новый столбец, можно назвать его "среднее значение прироста", это будет основной показатель для определения самого прибыльного вложения средств 
#"среднее значение прироста" определяется как произведение всех значений в строчке, из которого извлекается корень степени равной количеству значений 
df5 = pd.concat([df5, np.power(df4.prod(axis = 1),1/df5[0])], axis = 1, ignore_index=True)
#print(df5) 
df6=df5.copy()

def fun(df,v): #fun - рекурсивная функция выбирающая самое большое значение (по показателю "mean") из списка, 
               #записывает в датафрейм v, потом вызывает себя на участке слева и справа.
               #на входе предполагается датафрейм df5, на выходе перечисление строчек с максимальным приростом

    df.columns = ["len", "st", "mean"] # [длительность инвестирования, год, "средненее значение прироста"]
    df = df.sort_values(by=["mean"], ascending=False)
    v = pd.concat([v, df.iloc[0,:]],axis=1)

    #дальше надо выделить самый эффективный вариант вложения и, если возможно, вызвать эту же функцию слева и справа от себя
    if df[df['st']+df['len'] < df.iloc[0,1]+1].size > 0:
        df_l = df[df['st']+df['len'] < df.iloc[0,1]+1]
        v = fun(df_l, v)
#    else:
#        print('left empty')
    if df[df['st'] >=  df.iloc[0,1]+df.iloc[0,0]].size > 0: #справа 1 >= 0 + 1 у максимального
        df_r = df[df['st'] >=  df.iloc[0,1]+df.iloc[0,0]]
        v = fun(df_r, v)
#    else:
#        print('right empty')    
    return(v)

def enh(df, df6, v): #enhance - заполнение пустых клеток, за счет расширения времени инвестирования влево или вправо, если в "карте" есть подходящие значения.
                    # + возврат измененного дф6, из которого исключены итоговые значения v и пересекающиеся с ним.
                    # например если наиболее выгодно инвестировать в комп№1 в годы 10-13, а в комп №2 годы 15-17. Надо проверить год 14, если значение в нем >1 то он тоже был прибыльным для одной из компаний.

    v = v.sort_values(by="st", axis = 1)
    #сначала расширяем первое значение влево, если это возможно:
    end = int(v.iloc[1,0])                       #конец пропуска, который надо заполнить 
    st = int(0)                                  #начало пропуска, который надо заполнить
    check = [v.iloc[0,0] + (end - st),v.iloc[1,0] - (end - st)]

    #Условие для первой по времени компании под индексом v.columns.values.tolist()[0]:
    # Если произведение значений в диапазоне от начала таблицы до первого v "st" больше 1
    # И такой вариант есть в карте df6, то есть возможно выбрать:
        #Индекс (df6.index == v.columns.values.tolist()[0])
        #Новую точку начала (df6['st'] == v.iloc[1,0] - (end - st))
        #Новую длину (df6["len"] == v.iloc[0,0] + (end - st))
    if df.iloc[v.columns.values.tolist()[0],st:end].prod() > 1 \
        and (df6[df6.columns[:2]].loc[v.columns.values.tolist()[0]] == check).all(1).any():  
        #выполняется условие что выгоднее расширить v справа:значение больше 1, вариант есть в дф6 (в "карте")
        
        #print('до(1)', v)
        v.iloc[1,0] = v.iloc[1,0] - (end - st)
        v.iloc[0,0] = v.iloc[0,0] + (end - st)     
        #print('после(1)', v)

    for i in range(1, v.shape[1]):
        end = int(v.iloc[1,i])                       #конец пропуска, который надо заполнить 
        st = int(v.iloc[1,(i-1)]+v.iloc[0,(i-1)])    #начало пропуска, который надо заполнить 
        check1 = [v.iloc[0,i-1] + (end - st),v.iloc[1,i-1]] #значения для элемента i-1 если его можно ли его расширить вправо
        check2 = [v.iloc[0,i] + (end - st),v.iloc[1,i] - (end - st)] #значения для элемента i если его можно ли его расширить влево
             
        if df.iloc[v.columns.values.tolist()[i-1],st:end].prod() > 1 \
            and df.iloc[v.columns.values.tolist()[i-1],st:end].prod() > df.iloc[v.columns.values.tolist()[i],st:end].prod() \
            and (df6[df6.columns[:2]].loc[v.columns.values.tolist()[i-1]] == check1).all(1).any():
            #выполняется условие что выгоднее расширить v слева: значение больше 1, множитель больше чем если расширить справа, вариант есть в дф6 (в "карте")
            
            #print('до(2)', v)
            v.iloc[0,i-1] = v.iloc[0,i-1] + (end - st)        
            #print('после(2)', v)
        
        elif df.iloc[v.columns.values.tolist()[i],st:end].prod() > 1 \
            and (df6[df6.columns[:2]].loc[v.columns.values.tolist()[1]] == check2).all(1).any():

            #выполняется условие что выгоднее расширить v справа: значение больше 1, вариант есть в дф6 (в "карте")
            
            #print('до(3)', v) 
            v.iloc[1,i] = v.iloc[1,i] - (end - st)
            v.iloc[0,i] = v.iloc[0,i] + (end - st)     
            #print('после(3)', v)

    #расширяем последнее значение вправо, если это возможно:
    end = int(df.shape[1])                 #конец пропуска, который надо заполнить 
    st = int(v.iloc[1,-1]+v.iloc[0,-1])    #начало пропуска, который надо заполнить 
    check = [v.iloc[0,-1] + (end - st), v.iloc[1,-1]]

    if df.iloc[v.columns.values.tolist()[-1],st:end].prod() > 1 \
        and (df6[df6.columns[:2]].loc[v.columns.values.tolist()[-1]] == check).all(1).any():
        #выполняется условие что выгоднее расширить v слева: значение больше 1, вариант есть в дф6 (в "карте")
        
        #print('до(п)', v)
#        print(df6[(df6.index == v.columns.values.tolist()[-1]) & (df6['st'] == v.iloc[1,-1]) & (df6['len'] == v.iloc[0,-1] + (end - st))].any().all())
        v.iloc[0,-1] = v.iloc[0,-1] + (end - st)        
        #print('после(п)', v)

    #перебор и исключение из "карты" вариантов,пересекающихся с v
    for r in range(0, v.shape[1]):
        #отсекаем если начала отрезка меньше начала искомого, а конец больше начала искомого 
        df6 = df6[((df6.index == v.columns.values.tolist()[r]) & 
                   (df6['st'] < v.iloc[1,r] + v.iloc[0,r]) & 
                   (df6['st'] + df6['len'] > v.iloc[1,r])) == False] 
        #отсекаем если конец отрезка больше начала искомого, но начало меньше конца искомого
        df6 = df6[((df6.index == v.columns.values.tolist()[r]) & 
                   (df6['st'] + df6['len'] > v.iloc[1,r]) & 
                   (df6['st'] < v.iloc[1,r]+ v.iloc[0,r])) == False]
        #проверка - что осталось
#        print(df6[(df6.index == v.columns.values.tolist()[r])])

    return(df6, v)


In [ ]:
#@title Шаг 4. Определение n лучших вариантов инвестрования
n = 20 # число компаний между которыми равномерно будет распределен капитал. По умолчанию 20
print('n,y=', n,y)

df7 = df6.copy(deep=True) #backup
th = pd.DataFrame() #top holdings - лучшие варианты размещения, в индексе указаны имя и срок владения в формате "Имя(стартовый год,на срок)"

info = "Итоги инвестирования: \n" #для записи информации по итогам инвестировния
m0 = 0 #здесь будет суммироваться доходность

#В цикле поочередно будут выбраны n лучших вариантов размещения средств
for x in range(n):
    v = pd.DataFrame()
    v = fun(df6,v)  # лучший вариант в фрейме df6
    df6, v = enh(df, df6, v) # обновленный фрейм df6 за исключением лучшего. v измененен если это были выполнены условия
    #print('df6=',df6)
    
    th = pd.concat([th, v],axis=1)
    #print('df6.shape',df6.shape)
    
    name_str = ""
    name_str += "Позиция №" + str(x+1) + " с размещением: "
    m = 1 #показатель доходности по каждой позиции. Цифра 3 означает увеличение в 3 раза, т.е. +200%
    for i in range(0, v.shape[1]):
                                                             ### str(df_ind.iloc[v.columns.values.tolist()[i]])           
        name_str += str(v.columns.values.tolist()[i]) + "/" + str(df_ind.iloc[v.columns.values.tolist()[i]][0]) + " (столбец" + str(int(v.iloc[1,i])) + ", на срок" + str(int(v.iloc[0,i])) + "), "
        #print('i=',i)
        st = int(v.iloc[1,i])                 #конец пропуска, который надо заполнить 
        end = int(v.iloc[1,i]+v.iloc[0,i])    #начало пропуска, который надо заполнить 
        m = m * df.iloc[v.columns.values.tolist()[i],st:end].prod()
    name_str += "Доходность: " + str(m)
    
    info += name_str + ". \n"
    
    #print(name_str)
    #print(v)    
    #print('m=',m)
    m0 += m

###
if y < 3:
    nalog = np.power(0.87,th.shape[1]/n)
    info += 'Снижение прибыли по всему портфелю после налогообложения (по ставке 13%) составило:' + str((1-nalog)*100) + "% \n"
else:
    nalog = 1
    info += 'Из-за срока удержания акций 3 и более лет - прибыль не облагается налогом. \n'
info += 'Итоговая доходность по всему портфелю составила:' + str(m0/n*nalog) + ", годовой прирост: " + str(np.power(m0/n*nalog, 1/df.shape[1])) + ". \n"


th = th.drop(labels="mean")
print('th=',th)
#print('df6',df6)

df6 = df7 
#print('df6.shape2',df6.shape)

n,y= 20 4
th=       82   85   72   79   9    24   58   26   27   54  ...   10   81   23  \
len  4.0  4.0  4.0  6.0  4.0  6.0  4.0  7.0  4.0  4.0  ...  4.0  4.0  4.0   
st   3.0  7.0  1.0  5.0  1.0  5.0  0.0  4.0  1.0  5.0  ...  0.0  4.0  3.0   

      46   73   22   5    62   33   8   
len  4.0  4.0  4.0  4.0  4.0  4.0  7.0  
st   7.0  2.0  6.0  3.0  7.0  0.0  4.0  

[2 rows x 40 columns]


In [ ]:
#@title Шаг 5. Блок отображения данных
# на входе df_full и th (top holdings)
# Выводится:
# Таблица с именами компаний, без повторов. Периоды инвестирования выделяются зеленым
# Добавляются столбцы - капитализация в первый период инвестирования, средний дивидент за этот период

df_res = pd.DataFrame(df_full.loc["Цена скорр.2"].iloc[:,:-1])
df_res.index = df_res["Name"]

#df_ind['Name'].iloc[th.columns.unique()] - переводит номер из th в название компании
df_res = pd.DataFrame(df_res.loc[df_ind['Name'].iloc[th.columns.unique()]].iloc[:,-12:], dtype=float)

df_a_res = df_res.iloc[:,-11:]
df_b_res = df_res.iloc[:,-12:-1]
df_b_res.columns = df_a_res.columns
df_res = df_a_res.div(df_b_res).copy()

df_ind = df_ind.reset_index(drop=True)

count_cap_over10000, count_cap_over1000, count_cap_over100, count_cap_less100 = [0] * 4 # разделение на группы по капитализации
div_over10, div_over5, div_over05, div_less05  = [0] * 4 # разделение на группы по дивидентам

cap_list = []
div_list = []


for name in df_res.index:
    num = df_ind[df_ind['Name'] == name].index[0]
#    print(name)
    #print(th[num])
    
    z = th[num]
    
    #for (colname,colval) in z:
    #    print(colname, colval.values)
    
    i = 0
    cap = 0
    div = 0
    if z.size >3: ###перебор запускается если компания представлена в th больше 1 раза

        for index, values in z.iteritems():
#            print(values[0], values[1])
            st = int(values[1])
            end = int(values[1]+values[0])
                       
            for x in df_full[df_full['Name'] == name].loc["Капитализация, млрд руб"].iloc[-12+st:-12+end]:
                cap += float(x)
                i += 1
            for d in df_full[df_full['Name'] == name].loc["Див доход, ао, %"].iloc[-12+st:-12+end]:
                div += float(d)
                
    else:
#        print(z[0], z[1])
        st = int(z[1])
        end = int(z[1]+z[0])
                       
        for x in df_full[df_full['Name'] == name].loc["Капитализация, млрд руб"].iloc[-12+st:-12+end]:
            cap += float(x)
            i += 1
        for d in df_full[df_full['Name'] == name].loc["Див доход, ао, %"].iloc[-12+st:-12+end]:
            div += float(d)
    
#    print('Капитализация', name, cap/i)    
    if cap/i > 10000:
        count_cap_over10000 += 1
    elif cap/i > 1000:
        count_cap_over1000 += 1
    elif cap/i > 100:
        count_cap_over100 += 1
    else:
        count_cap_less100 += 1
        
#    print('Дивидент', name, div/i)   
    if div/i > 0.1:
        div_over10 += 1
    elif div/i > 0.05:
        div_over5 += 1
    elif div/i > 0.005:
        div_over05 += 1
    else:
        div_less05 += 1

    cap_list.append(cap/i)  
    div_list.append(div/i*100)      
        
df_cap = pd.DataFrame(cap_list, index = df_res.index, columns = ["Капитализация, средн. (млрд руб)"])
df_div = pd.DataFrame(div_list, index = df_res.index, columns = ["Дивиденды, средн. (%)"])        
df_info = pd.concat([df_cap,df_div], axis = 1)        
        
        
print(info)
print('Капитализация - количество компаний (больше 10трлн, 10-1трлн, 1000-100млрд, меньше 100млрд):',count_cap_over10000, count_cap_over1000, count_cap_over100, count_cap_less100)
print('Дивиденды - количество компаний (больше 10%, 10-5%, 5-0.5%, меньше):',div_over10, div_over5, div_over05, div_less05)


#Далее справочная информация об инфляции и изменении индекса ММВБ за период

inf = [20.1, 18.8, 15.06, 11.99, 11.74, 10.91, 9, 11.87, 13.28, 8.8, 8.78, 6.1, 6.58, 6.45, 11.36, 12.9, 5.4, 2.5, 4.3, 3, 4.9, 8.4, 11.9]
df_inf = pd.DataFrame(np.subtract([100]*23,inf)/100, index=[str(x) for x in range(2000,2023)])
def inflation(year1,year2): #функция показывает инфляцию за период, нужно учесть что первый год включать не следует
    return round(((1 - df_inf.loc[year1:year2].prod().values[0])* 100),1)

print('Инфляция за период c {} по {} составила {}%'.format(df_res.columns[0],df_res.columns[-1],inflation(df_res.columns[1],df_res.columns[-1])))

# https://www.moex.com/ru/index/totalreturn/MCFTR/archive  
# Индекс полной доходности ММВБ ("нетто")
data = pd.read_csv("/content/drive/My Drive/Colab Notebooks/archive_indices_2000-01-01-2023-03-03.csv", sep=';', header=0, index_col=0, encoding='latin-1', decimal=',')
# преобразование столбца с датой в формат datetime
data['TRADEDATE'] = pd.to_datetime(data['TRADEDATE'], format='%d.%m.%Y')
# группировка данных по годам и нахождение среднего значения
grouped_data = data.groupby(data['TRADEDATE'].dt.year)['OPEN'].mean()
# отбрасываем первое и последнее значение - неполные года
MCFTRR = grouped_data[1:-1]

def imoex(year1,year2): #функция показывает изменение индекса ММВБ(полной доходности), нужно учесть что первый год включать не следует
    return round(MCFTRR[int(year2)]/MCFTRR[int(year1)],2) 

print('Изменение индекса ММВБ(полной доходности,"нетто") за период c {} по {} составило:{}'.format(df_res.columns[0],df_res.columns[-1],imoex(df_res.columns[0],df_res.columns[-1])))


def highlight_cells(df_res):
    df10 = df_res.copy()
    df10.loc[:,:] = ''
    for r in range(0, th.shape[1]):
        i = df_ind['Name'].iloc[th.columns.values.tolist()[r]]
        df10.loc[i,"%s" % int(2011+th.iloc[1,r]):"%s" % int(2011+th.iloc[1,r]+ th.iloc[0,r]-1)] = 'background-color: #00EE00'
    return df10   


def bold(val):
    bold = 'bold' if float(val) > 1 else ''
    return 'font-weight: %s' % bold

t = pd.DataFrame() 
df_res = pd.concat([df_res,df_info], axis = 1)     


t = df_res.style.\
    apply(highlight_cells, axis=None).\
    applymap(bold)

#Итог
t

Итоги инвестирования: 
Позиция №1 с размещением: 82/Казаньоргсинтез (с года3, на срок4), 85/Селигдар (с года7, на срок4), Доходность: 102.88751595016733. 
Позиция №2 с размещением: 72/ФосАгро (с года1, на срок4), 79/Тинькофф Банк (с года5, на срок6), Доходность: 203.3779598735595. 
Позиция №3 с размещением: 9/Магнит (с года1, на срок4), 24/МГТС (с года5, на срок6), Доходность: 39.34978432162382. 
Позиция №4 с размещением: 58/Транснефть (с года0, на срок4), 26/НМТП (с года4, на срок7), Доходность: 35.29870218770808. 
Позиция №5 с размещением: 27/М.Видео (с года1, на срок4), 54/Россети Волга (с года5, на срок4), Доходность: 20.04826516867247. 
Позиция №6 с размещением: 2/НКНХ (с года1, на срок4), 83/Распадская (с года5, на срок6), Доходность: 43.40222542027941. 
Позиция №7 с размещением: 31/ММК (с года3, на срок4), 14/Таттелеком (с года7, на срок4), Доходность: 36.824525540829804. 
Позиция №8 с размещением: 81/Татнефть (с года0, на срок4), 71/ИнтерРАО (с года4, на срок7), Доходность: 11.

<ipython-input-28-83a16beb3d47>:42: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for index, values in z.iteritems():


,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,"Капитализация, средн. (млрд руб)","Дивиденды, средн. (%)"
Name,,,,,,,,,,,,,
Казаньоргсинтез,nan,1.317250,1.428987,2.088136,2.490071,1.401504,2.012376,1.340936,1.109644,0.811460,1.234543,76.525000,8.500000
Селигдар,nan,nan,nan,nan,nan,1.824351,1.090810,0.997994,1.236181,4.119764,1.380404,31.425000,8.900000
ФосАгро,nan,2.431976,0.819969,1.792834,1.753213,0.998157,1.023312,1.051108,1.018765,1.395282,1.996291,216.025000,5.475000
Тинькофф Банк,nan,nan,nan,nan,nan,2.938476,1.748116,1.025390,1.285840,1.829554,2.618539,416.633333,3.033333
Магнит,0.694340,1.704334,1.955541,1.065266,1.197318,1.007409,0.596068,0.585022,1.061150,1.773033,1.046129,826.400000,2.425000
МГТС,nan,nan,nan,nan,nan,2.330181,2.042011,1.441250,1.286776,1.154639,0.908482,156.700000,9.850000
Транснефть,1.312343,1.391431,1.253916,1.455991,1.539683,1.020361,0.908815,1.012945,1.097685,0.854432,1.210422,587.925000,0.000000
НМТП,0.779044,0.902863,1.000954,0.415534,3.667638,1.752660,1.407032,0.872611,1.528619,1.043576,0.840989,134.514286,8.871429
М.Видео,0.692509,1.570030,1.289788,0.520858,2.844368,1.533601,1.070968,0.987952,1.264878,1.524018,0.655089,41.950000,17.575000
